На вход подается синусоида произвольной длины, случайным значением амплитуды и частоты. Задача: по входной последовательности определить значения аплитуды и частоты

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
device = 'cpu'

In [2]:
def gen_data(batch = 4):
    t = np.arange(0,torch.randint(3,15,(1,)),step=0.1)
    X = torch.zeros(batch,t.size,1)
    Y = torch.zeros(batch,2)
    for i in range(batch):
        a = torch.rand(1)*3 + 0.1
        b = torch.rand(1)*4 + 0.2
        X[i,:,0] =  a*np.sin(b*t)
        Y[i,0],Y[i,1] = a,b
    return X,Y

In [3]:
from torch.nn import TransformerEncoder, TransformerEncoderLayer
class TransformerRegression(nn.Module):
    def __init__(self, input_dim, output_dim, nhead, hidden_dim, num_layers):
        super(TransformerRegression, self).__init__()

        self.embedding = nn.Linear(input_dim, hidden_dim)
        encoder_layer = TransformerEncoderLayer(hidden_dim, nhead)
        self.encoder = TransformerEncoder(encoder_layer, num_layers)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2)  # Изменяем форму для передачи в трансформер
        output = self.encoder(x)  # Применяем трансформер
        output = output[0,:,:]
        output = self.fc(output)  # Полносвязный слой для предсказания
        return output

In [4]:

input_dim = 1
output_dim = 2
nhead = 2#4
hidden_dim = 32
num_layers = 2#4

model = TransformerRegression(input_dim, output_dim, nhead, hidden_dim, num_layers)

/home/strike/penv/deep/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
s = []
# Обучение модели
num_epochs = 4000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    x,y = gen_data(batch=16)
    output = model(x.to(device))
    
    loss = criterion(output, y.to(device))  # Используем только первый измерение для задачи регрессии
    loss.backward()
    optimizer.step()
    l = loss.detach().to('cpu')
    s.append(l)
    if (epoch + 1) % 100 == 0:
        L = np.mean(s)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {L:.4f}')
        s = []

/home/strike/penv/deep/lib/python3.10/site-packages/torch/autograd/__init__.py:266: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch [100/4000], Loss: 0.8408
Epoch [200/4000], Loss: 0.6741
Epoch [300/4000], Loss: 0.5608
Epoch [400/4000], Loss: 0.5582
Epoch [500/4000], Loss: 0.4994
Epoch [600/4000], Loss: 0.5361
Epoch [700/4000], Loss: 0.4854
Epoch [800/4000], Loss: 0.4851
Epoch [900/4000], Loss: 0.5104
Epoch [1000/4000], Loss: 0.4741
Epoch [1100/4000], Loss: 0.4830
Epoch [1200/4000], Loss: 0.4789
Epoch [1300/4000], Loss: 0.4796
Epoch [1400/4000], Loss: 0.4936
Epoch [1500/4000], Loss: 0.4540
Epoch [1600/4000], Loss: 0.4794
Epoch [1700/4000], Loss: 0.4864
Epoch [1800/4000], Loss: 0.4782
Epoch [1900/4000], Loss: 0.4733
Epoch [2000/4000], Loss: 0.4563
Epoch [2100/4000], Loss: 0.5110
Epoch [2200/4000], Loss: 0.4845
Epoch [2300/4000], Loss: 0.5186
Epoch [2400/4000], Loss: 0.4835
Epoch [2500/4000], Loss: 0.4910
Epoch [2600/4000], Loss: 0.4674
Epoch [2700/4000], Loss: 0.4591
Epoch [2800/4000], Loss: 0.4596
Epoch [2900/4000], Loss: 0.4789
Epoch [3000/4000], Loss: 0.4577
Epoch [3100/4000], Loss: 0.4657
Epoch [3200/4000]

Ошибка на тесте

In [15]:
x,y = gen_data(batch=32)
output = model(x)
loss = criterion(output, y.to(device))
print(loss.detach())

tensor(0.4883)


Добавим позиционное кодирование

In [16]:
from torch.nn import TransformerEncoder, TransformerEncoderLayer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(torch.log(torch.tensor(10000.0)) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        return x + self.encoding[:, :x.size(1)].detach()

class TransformerRegression(nn.Module):
    def __init__(self, input_dim, output_dim, nhead, hidden_dim, num_layers):
        super(TransformerRegression, self).__init__()
        self.pos_enc = PositionalEncoding(hidden_dim,1000)
        self.embedding = nn.Linear(input_dim, hidden_dim)
        encoder_layer = TransformerEncoderLayer(hidden_dim, nhead)
        self.encoder = TransformerEncoder(encoder_layer, num_layers)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_enc(x)
        x = x.permute(1, 0, 2)  # Изменяем форму для передачи в трансформер
        output = self.encoder(x)  # Применяем трансформер
        output = output[0,:,:]
        output = self.fc(output)  # Полносвязный слой для предсказания
        return output

In [17]:
model_pos_cod = TransformerRegression(input_dim, output_dim, nhead, hidden_dim, num_layers)

/home/strike/penv/deep/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [18]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model_pos_cod.parameters(), lr=0.001)
s = []
# Обучение модели
num_epochs = 4000
for epoch in range(num_epochs):
    model_pos_cod.train()
    optimizer.zero_grad()

    x,y = gen_data(batch=16)
    output = model_pos_cod(x.to(device))
    
    loss = criterion(output, y.to(device))  # Используем только первый измерение для задачи регрессии
    loss.backward()
    optimizer.step()
    l = loss.detach().to('cpu')
    s.append(l)
    if (epoch + 1) % 100 == 0:
        L = np.mean(s)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {L:.4f}')
        s = []

Epoch [100/4000], Loss: 0.8334
Epoch [200/4000], Loss: 0.3511
Epoch [300/4000], Loss: 0.0973
Epoch [400/4000], Loss: 0.0653
Epoch [500/4000], Loss: 0.0615
Epoch [600/4000], Loss: 0.0705
Epoch [700/4000], Loss: 0.0463
Epoch [800/4000], Loss: 0.0375
Epoch [900/4000], Loss: 0.0335
Epoch [1000/4000], Loss: 0.0496
Epoch [1100/4000], Loss: 0.0299
Epoch [1200/4000], Loss: 0.0251
Epoch [1300/4000], Loss: 0.0251
Epoch [1400/4000], Loss: 0.0249
Epoch [1500/4000], Loss: 0.0392
Epoch [1600/4000], Loss: 0.0203
Epoch [1700/4000], Loss: 0.0460
Epoch [1800/4000], Loss: 0.0268
Epoch [1900/4000], Loss: 0.0194
Epoch [2000/4000], Loss: 0.0210
Epoch [2100/4000], Loss: 0.0220
Epoch [2200/4000], Loss: 0.0180
Epoch [2300/4000], Loss: 0.0159
Epoch [2400/4000], Loss: 0.0147
Epoch [2500/4000], Loss: 0.0160
Epoch [2600/4000], Loss: 0.0229
Epoch [2700/4000], Loss: 0.0156
Epoch [2800/4000], Loss: 0.0135
Epoch [2900/4000], Loss: 0.0147
Epoch [3000/4000], Loss: 0.0121
Epoch [3100/4000], Loss: 0.0106
Epoch [3200/4000]

Ошибка на тесте

In [19]:
x,y = gen_data(batch=32)
output = model_pos_cod(x)
loss = criterion(output, y.to(device))
print(loss.detach())

tensor(0.0135)
